In [1]:
import pandas as pd

from data_transformations_ps import Tickets

ImportError: cannot import name 'Subscriptions' from 'data_transformations_ps.subscriptions.subscriptions' (/mnt/c/Users/bkessler/Desktop/packages/data_transformations_ps/subscriptions/subscriptions.py)

In [2]:
ticketing = Tickets(fys=[2019])

In [3]:
ticketing.convert_dates().drop_unsold().add_dow().filter_series(['Classics'])\
         .transform_price_type_group().add_concert_numbers_clx()

Tickets(fys='[2019]')

In [46]:
df = ticketing.data.groupby(['concert_number', 'dow', 'price_type_group']).agg({
    'paid_amt': ['sum', 'count']
}).unstack('price_type_group').reset_index()

In [56]:
df[('paid_amt', 'sum', 'Total')] = df[('paid_amt', 'sum', 'Single')] + df[('paid_amt', 'sum', 'Subscription')]
df[('paid_amt', 'count', 'Total')] = df[('paid_amt', 'count', 'Single')] + df[('paid_amt', 'count', 'Subscription')]
df['capacity_sold'] = df[('paid_amt', 'count', 'Total')] / 1750

In [58]:
df.columns = ['concert_number', 'dow', 'single_rev', 'sub_rev', 'single_units', 'sub_units', 'total_rev', 'total_units', 'capacity_sold']

In [93]:
df = df.melt(id_vars=['concert_number', 'dow'], 
               value_vars=['single_rev', 'sub_rev', 'single_units', 'sub_units', 
                           'total_rev', 'total_units', 'capacity_sold'])

In [96]:
df = df.groupby(['concert_number', 'variable', 'dow']).sum().unstack('dow').reset_index()

In [97]:
sort_mapper = {
    'single_rev': 1,
    'single_units': 2,
    'sub_rev': 3,
    'sub_units': 4,
    'total_rev': 5,
    'total_units': 6,
    'capacity_sold': 7
}

In [99]:
df['sorter'] = df['variable'].map(sort_mapper)

In [102]:
df = df.sort_values(by=['concert_number', 'sorter']).reset_index(drop=True)

In [103]:
df

concert_number       variable         value                              \
dow                                      Friday      Saturday      Thursday   
0                1     single_rev  22384.250000  34834.500000  31518.500000   
1                1   single_units    377.000000    473.000000    464.000000   
2                1        sub_rev  43124.000000  49826.000000  53044.500000   
3                1      sub_units    685.000000    792.000000    762.000000   
4                1      total_rev  65508.250000  84660.500000  84563.000000   
..             ...            ...           ...           ...           ...   
79              12        sub_rev  43025.200000  47022.350000  41833.300000   
80              12      sub_units    627.000000    706.000000    596.000000   
81              12      total_rev  53838.950000  66080.850000  51825.050000   
82              12    total_units    873.000000   1059.000000    802.000000   
83              12  capacity_sold      0.498857      0.605143      0.458286   

            sorter  
dow Tuesday         
0       NaN      1  
1       NaN      2  
2       NaN      3  
3       NaN      4  
4       NaN      5  
..      ...    ...  
79      NaN      3  
80      NaN      4  
81      NaN      5  
82      NaN      6  
83      NaN      7  

[84 rows x 7 columns]

In [104]:
df.to_excel('dow_2019_clx.xlsx')